# PyCity Schools Analysis

* PyCity has 15 Charter and District schools, that serve 39170 students. 
* Charter schools outperformed district schools in overall passing rates across all grades, with a 95.10% overall passing rate, compared to district schools with only 73.67%.
* The Charter School: Cabrera Highschool having a 95.59% passing rate for both math and reading. In Contrast the District School: Rodriguez High School only had a 73.29% passing rate both math and reading.  
* In terms of school size, schools with high enrollment had a much lower overall passing rate (76.36%) than small (94.82%) and medium (95.20%) schools. 

In [16]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.tail()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95,75,14,Charter,1635,1043130


## District Summary

In [17]:
# Calculate the total number of unique schools
school_count = school_data_complete['school_name'].nunique()
print("There are "+str(school_count)+" schools in the district.")

There are 15 schools in the district.


In [20]:
# Calculate the total number of students
student_count = school_data_complete['Student ID'].count()
print("there are "+str(student_count)+" students in this district")

there are 39170 students in this district


In [23]:
# Calculate the total budget
total_budget = school_data['budget'].sum()
print(total_budget)

24649428


In [26]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean().__round__(2)
print(average_math_score)

78.99


In [27]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean().__round__(2)
print(average_reading_score)

81.88


In [31]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = round(passing_math_count / float(student_count) * 100)
passing_math_percentage

75

In [32]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  

passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = round(passing_reading_count / float(student_count) * 100)
passing_reading_percentage

86

In [56]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate = overall_passing_rate.round(2)
overall_passing_rate

65.17

In [57]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [average_reading_score],
    "Average Math Score": [average_math_score],
    "% Passing Reading":[passing_reading_percentage],
    "% Passing Math": [passing_math_percentage],
    "Overall Passing Rate": [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,"39,170","$24,649,428.00",81.88,78.99,86,75,65.17


## School Summary

In [58]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [59]:
# Calculate the total student count
per_school_counts = school_data_complete.groupby(['school_name']).nunique()["student_name"]
per_school_counts.head()

school_name
Bailey High School      4799
Cabrera High School     1834
Figueroa High School    2886
Ford High School        2676
Griffin High School     1449
Name: student_name, dtype: int64

In [60]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = (per_school_budget / per_school_counts).round(2)
per_school_capita.head()

/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/1678131222.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]


school_name
Bailey High School      651.16
Cabrera High School     589.62
Figueroa High School    652.95
Ford High School        659.16
Griffin High School     633.20
dtype: float64

In [86]:
# Calculate the average test scores by school
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"].round(2)
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"].round(2)

per_school_combined = (per_school_math + per_school_reading)/2

per_school_combined = per_school_combined.round(2)

per_school_combined

/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/96219715.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"].round(2)
/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/96219715.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"].round(2)


school_name
Bailey High School       79.04
Cabrera High School      83.52
Figueroa High School     78.94
Ford High School         78.92
Griffin High School      83.58
Hernandez High School    79.11
Holden High School       83.80
Huang High School        78.90
Johnson High School      79.02
Pena High School         83.94
Rodriguez High School    78.79
Shelton High School      83.54
Thomas High School       83.64
Wilson High School       83.63
Wright High School       83.82
dtype: float64

In [65]:
# Calculate the number of students per schools with math scores of 70 or higher
school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["student_name"].count()

school_passing_math


school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: student_name, dtype: int64

In [88]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["student_name"].count()

school_passing_reading

school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: student_name, dtype: int64

In [87]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

passing_math_and_reading.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


In [89]:
# Use the provided code to calculate the passing rates

per_school_counts = school_data_complete["school_name"].value_counts()

school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["student_name"].count()
per_school_passing_math = school_passing_math / per_school_counts * 100


school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["student_name"].count()

per_school_passing_reading = school_passing_reading / per_school_counts * 100

overall_passing_rate = ((per_school_passing_math + per_school_passing_reading) / 2).round(2)

#per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
#overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

#overall_passing_rate

overall_passing_rate

Bailey High School       74.31
Cabrera High School      95.59
Figueroa High School     73.36
Ford High School         73.80
Griffin High School      95.27
Hernandez High School    73.81
Holden High School       94.38
Huang High School        73.50
Johnson High School      73.64
Pena High School         95.27
Rodriguez High School    73.29
Shelton High School      94.86
Thomas High School       95.29
Wilson High School       95.20
Wright High School       94.97
dtype: float64

In [148]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.

per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": school_passing_math,
    "% Passing Reading": school_passing_reading,
    "% Passing Overall": overall_passing_rate
})


# Formatting
#per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
#per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,District,4976,3124928.0,651.16,77.05,81.03,3318,4077,74.31
Cabrera High School,Charter,1858,1081356.0,589.62,83.06,83.98,1749,1803,95.59
Figueroa High School,District,2949,1884411.0,652.95,76.71,81.16,1946,2381,73.36
Ford High School,District,2739,1763916.0,659.16,77.10,80.75,1871,2172,73.80
Griffin High School,Charter,1468,917500.0,633.20,83.35,83.82,1371,1426,95.27
Hernandez High School,District,4635,3022020.0,671.41,77.29,80.93,3094,3748,73.81
Holden High School,Charter,427,248087.0,585.11,83.80,83.81,395,411,94.38
Huang High School,District,2917,1910635.0,668.52,76.63,81.18,1916,2372,73.50
Johnson High School,District,4761,3094650.0,670.13,77.07,80.97,3145,3867,73.64
Pena High School,Charter,962,585858.0,613.46,83.84,84.04,910,923,95.27


## Highest-Performing Schools (by % Overall Passing)

In [149]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.

highest_performing_schools_passing_rate = per_school_summary.sort_values(by="% Passing Overall", ascending=False)

highest_performing_schools_passing_rate.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,1858,1081356.0,589.62,83.06,83.98,1749,1803,95.59
Thomas High School,Charter,1635,1043130.0,644.31,83.42,83.85,1525,1591,95.29
Griffin High School,Charter,1468,917500.0,633.20,83.35,83.82,1371,1426,95.27
Pena High School,Charter,962,585858.0,613.46,83.84,84.04,910,923,95.27
Wilson High School,Charter,2283,1319574.0,587.00,83.27,83.99,2143,2204,95.20


## Bottom Performing Schools (By % Overall Passing)

In [150]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.

lowest_performing_schools_passing_rate = per_school_summary.sort_values(by="% Passing Overall", ascending=True)

lowest_performing_schools_passing_rate.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,3999,2547363.0,652.84,76.84,80.74,2654,3208,73.29
Figueroa High School,District,2949,1884411.0,652.95,76.71,81.16,1946,2381,73.36
Huang High School,District,2917,1910635.0,668.52,76.63,81.18,1916,2372,73.50
Johnson High School,District,4761,3094650.0,670.13,77.07,80.97,3145,3867,73.64
Ford High School,District,2739,1763916.0,659.16,77.10,80.75,1871,2172,73.80


## Math Scores by Grade

In [151]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_math_scores = ninth_graders.groupby("school_name")["math_score"].mean().round(2)
tenth_graders_math_scores = tenth_graders.groupby("school_name")["math_score"].mean().round(2)
eleventh_graders_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean().round(2)
twelfth_graders_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean().round(2)

# Use the code to select only the `math_score`.
#ninth_grade_math_scores = ninth_graders_scores["math_score"]
#tenth_grader_math_scores = tenth_graders_scores["math_score"]
#eleventh_grader_math_scores = eleventh_graders_scores.mean()["math_score"]
#twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`

math_scores_by_grade = pd.DataFrame({
    "9th": ninth_graders_math_scores,
    "10th": tenth_graders_math_scores,
    "11th": eleventh_graders_math_scores,
    "12th": twelfth_graders_math_scores
})


# Minor data wrangling
#math_scores_by_grade.index.name = None

# Display the DataFrame
#math_scores_by_grade

#ninth_graders_scores

math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [152]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]


ninth_graders_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean().round(2)
tenth_graders_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean().round(2)
eleventh_graders_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean().round(2)
twelfth_graders_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean().round(2)

# Group by "school_name" and take the mean of each.
#ninth_graders_scores = 
#tenth_graders_scores = 
#eleventh_graders_scores = 
#twelfth_graders_scores = 

# Use the code to select only the `reading_score`.
#ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
#tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
#eleventh_grader_reading_scores = eleventh_graders_scores.mean()["reading_score"]
#twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`

reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_graders_reading_scores,
    "10th": tenth_graders_reading_scores,
    "11th": eleventh_graders_reading_scores,
    "12th": twelfth_graders_reading_scores
})

# Minor data wrangling
#reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
#reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [173]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [174]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [178]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins=spending_bins, labels=labels)

school_spending_df.sort_values("Spending Ranges (Per Student)", ascending=True)

school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,School Size,Spending Ranges (Per Student)
Bailey High School,District,4976,3124928.0,651.16,77.05,81.03,3318,4077,74.31,Large (2000-5000),$645-680
Cabrera High School,Charter,1858,1081356.0,589.62,83.06,83.98,1749,1803,95.59,Medium (1000-2000),$585-630
Figueroa High School,District,2949,1884411.0,652.95,76.71,81.16,1946,2381,73.36,Large (2000-5000),$645-680
Ford High School,District,2739,1763916.0,659.16,77.10,80.75,1871,2172,73.80,Large (2000-5000),$645-680
Griffin High School,Charter,1468,917500.0,633.20,83.35,83.82,1371,1426,95.27,Medium (1000-2000),$630-645
Hernandez High School,District,4635,3022020.0,671.41,77.29,80.93,3094,3748,73.81,Large (2000-5000),$645-680
Holden High School,Charter,427,248087.0,585.11,83.80,83.81,395,411,94.38,Small (<1000),$585-630
Huang High School,District,2917,1910635.0,668.52,76.63,81.18,1916,2372,73.50,Large (2000-5000),$645-680
Johnson High School,District,4761,3094650.0,670.13,77.07,80.97,3145,3867,73.64,Large (2000-5000),$645-680
Pena High School,Charter,962,585858.0,613.46,83.84,84.04,910,923,95.27,Small (<1000),$585-630


In [156]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"].round(2)
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"].round(2)
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"].round(2)
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"].round(2)
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Overall"].round(2)

/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/4252705949.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"].round(2)
/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/4252705949.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"].round(2)
/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/4252705949.py:4: FutureWarning: The defau

In [157]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,NaN,NaN,NaN,NaN,NaN
$585-630,83.50,83.92,1421.67,1461.33,95.04
$630-645,83.38,83.84,1448.00,1508.50,95.28
$645-680,76.96,80.97,2563.43,3117.86,73.67


## Scores by School Size

In [158]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [159]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=labels)

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,School Size
Bailey High School,District,4976,3124928.0,651.16,77.05,81.03,3318,4077,74.31,Large (2000-5000)
Cabrera High School,Charter,1858,1081356.0,589.62,83.06,83.98,1749,1803,95.59,Medium (1000-2000)
Figueroa High School,District,2949,1884411.0,652.95,76.71,81.16,1946,2381,73.36,Large (2000-5000)
Ford High School,District,2739,1763916.0,659.16,77.10,80.75,1871,2172,73.80,Large (2000-5000)
Griffin High School,Charter,1468,917500.0,633.20,83.35,83.82,1371,1426,95.27,Medium (1000-2000)
Hernandez High School,District,4635,3022020.0,671.41,77.29,80.93,3094,3748,73.81,Large (2000-5000)
Holden High School,Charter,427,248087.0,585.11,83.80,83.81,395,411,94.38,Small (<1000)
Huang High School,District,2917,1910635.0,668.52,76.63,81.18,1916,2372,73.50,Large (2000-5000)
Johnson High School,District,4761,3094650.0,670.13,77.07,80.97,3145,3867,73.64,Large (2000-5000)
Pena High School,Charter,962,585858.0,613.46,83.84,84.04,910,923,95.27,Small (<1000)


In [160]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"].round(2)
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"].round(2)
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"].round(2)
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"].round(2)
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Passing Overall"].round(2)

/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/2305847913.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"].round(2)
/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/2305847913.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"].round(2)
/var/folders/m0/xq3r3nm57yq55crv150tzx9h0000gn/T/ipykernel_91877/2305847913.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy

In [161]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`

size_summary = pd.DataFrame({
    "Average Math Scores": size_math_scores,
    "Average Reading Scores": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})

# Display results
size_summary

,Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,652.50,667.00,94.82
Medium (1000-2000),83.37,83.87,1595.60,1649.40,95.20
Large (2000-5000),77.74,81.34,2510.88,3003.62,76.36


## Scores by School Type

In [168]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby("School Type")["Average Math Score"].mean().round(2)
type_reading_scores = per_school_summary.groupby("School Type")["Average Reading Score"].mean().round(2)
type_passing_math = per_school_summary.groupby("School Type")["% Passing Math"].mean().round(2)
type_passing_reading = per_school_summary.groupby("School Type")["% Passing Reading"].mean().round(2) 
type_overall_passing = per_school_summary.groupby("School Type")["% Passing Overall"].mean().round(2)

# Use the code provided to select new column data
#average_math_score_by_type = type_math_scores["Average Math Score"]
#average_reading_score_by_type = type_reading_scores["Average Reading Score"]
#average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
#average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
#average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]


In [169]:
# Assemble the new data by type into a DataFrame called `type_summary`

type_summary = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Passing Overall": type_overall_passing
})

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47,83.90,1428.25,1473.12,95.10
District,76.96,80.97,2563.43,3117.86,73.67
